In [ ]:
import pyodbc
import getpass
import pandas as pd
from os import path
import time

In [ ]:
dir_data = '/path/to/data/dir'
file_concepts = path.join(dir_data, 'concepts2.csv')
file_persons = path.join(dir_data, 'persons.csv')
file_ingredients = path.join(dir_data, 'drug_ingredients.csv')
file_condition_ancestors = path.join(dir_data, 'condition_ancestors.csv')

# SQL server config
sql_config = {
    'driver': '',
    'server': '',
    'database': '',
    'uid': ''
}

In [ ]:
pwd=getpass.getpass()

In [ ]:
conn = pyodbc.connect(**sql_config, pwd=pwd)
cursor = conn.cursor()

# Get Metadata

### Get concept definitions

In [ ]:
sql = """SELECT concept_id, concept_name, domain_id, vocabulary_id, concept_class_id, standard_concept, concept_code, valid_start_date, valid_end_date, invalid_reason
    FROM concept;"""
df_concpets = pd.read_sql(sql, conn)
df_concpets.to_csv(file_concepts, sep='\t', na_rep='NULL', header=True, index=False)

### Get ingredients for each drug

In [ ]:
sql = """SELECT drug_concept_id, ancestor_concept_id
FROM 
    (SELECT DISTINCT drug_concept_id
    FROM drug_exposure) de
JOIN concept_ancestor ca ON ca.descendant_concept_id = de.drug_concept_id
JOIN concept c_a ON ca.ancestor_concept_id = c_a.concept_id
WHERE c_a.concept_class_id = 'Ingredient';"""
df_ingredients = pd.read_sql(sql, conn)
df_ingredients.to_csv(file_ingredients, sep='\t', na_rep='NULL', header=True, index=False)
df_ingredients.head(10)

### Get ancestors for concepts
Only use concepts that already have observations so that we don't introduce a large new set
Keep only up to a max_levels of 3 so that observed concepts don't contribute too far up the chain

In [ ]:
sql = """ SELECT DISTINCT condition_concept_id
INTO #observed_conditions
FROM condition_occurrence;"""
cursor.execute(sql)

sql = """SELECT ocd.condition_concept_id, ca.ancestor_concept_id
FROM #observed_conditions ocd
JOIN concept_ancestor ca ON ca.descendant_concept_id = ocd.condition_concept_id
JOIN #observed_conditions oca ON ca.ancestor_concept_id = oca.condition_concept_id
WHERE ca.min_levels_of_separation > 0 AND ca.max_levels_of_separation <= 2;"""
df_condition_ancestors = pd.read_sql(sql, conn)
df_condition_ancestors.to_csv(file_condition_ancestors, sep='\t', na_rep='NULL', header=True, index=False)
df_condition_ancestors.head(10)

# Get patient data

### Patient data extraction without data cleaning (default)

In [ ]:
sql = """SELECT person_id, gender_concept_id, CONVERT(DATE, birth_datetime) AS birth_date, race_concept_id, ethnicity_concept_id 
    FROM person"""
df_persons = pd.read_sql(sql, conn)
df_persons.to_csv(file_persons, sep='\t', na_rep='NULL', header=True, index=False)
n_persons = len(df_persons.index)

### Patient data extraction with data cleaning

In [ ]:
# Get the highest ancestors of race_concept_id because our database has many asians recorded by their nationality
# Also exclude patients whose birth_datetime is a known bad birth_datetime
sql = """SELECT person_id, gender_concept_id, CONVERT(DATE, p.birth_datetime) AS birth_date, r.ancestor_concept_id AS race_concept_id, ethnicity_concept_id 
    FROM person p
    JOIN user_schema.dbo.race_ancestor_concepts r ON p.race_concept_id = r.race_concept_id
    LEFT JOIN user_schema.dbo.ohdsi_2019q2_bad_dobs b ON p.birth_datetime = b.birth_datetime
    WHERE b.birth_datetime IS NULL AND p.year_of_birth >= 1900;"""

df_persons = pd.read_sql(sql, conn)
df_persons.to_csv(file_persons, sep='\t', na_rep='NULL', header=True, index=False)
n_persons = len(df_persons.index)

In [ ]:
df_persons.columns

## Get sequence for each patient

In [ ]:
# Read df_persons back in from file
df_persons = pd.read_csv(file_persons, sep='\t', header=0)
n_persons = len(df_persons.index)

In [ ]:
t1 = time.time()

sql = """SELECT x.*
FROM 
    ((SELECT co.condition_concept_id AS concept_id, MIN(co.condition_start_date) AS start_date
    FROM dbo.condition_occurrence co
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON co.condition_concept_id = i.concept_id
    JOIN concept c ON co.condition_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON co.visit_occurrence_id = v.visit_occurrence_id
    WHERE co.person_id = ? AND co.condition_concept_id != 0 AND i.concept_id IS NULL AND co.condition_start_date >= ? 
        AND c.domain_id = 'Condition' AND (v.visit_occurrence_id IS NULL OR co.condition_start_date <= v.visit_end_date)
    GROUP BY condition_concept_id)
    UNION
    (SELECT do.drug_concept_id AS concept_id, MIN(do.drug_exposure_start_date) AS start_date
    FROM dbo.drug_exposure do
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON do.drug_concept_id = i.concept_id
    JOIN concept c ON do.drug_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON do.visit_occurrence_id = v.visit_occurrence_id
    WHERE do.person_id = ? AND do.drug_concept_id != 0 AND i.concept_id IS NULL AND do.drug_exposure_start_date >= ? 
        AND c.domain_id = 'Drug' AND (v.visit_occurrence_id IS NULL OR do.drug_exposure_start_date <= v.visit_end_date)
    GROUP BY drug_concept_id)
    UNION
    (SELECT po.procedure_concept_id AS concept_id, MIN(po.procedure_date) AS start_date
    FROM dbo.procedure_occurrence po
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON po.procedure_concept_id = i.concept_id
    JOIN concept c ON po.procedure_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON po.visit_occurrence_id = v.visit_occurrence_id
    WHERE po.person_id = ? AND po.procedure_concept_id != 0 AND i.concept_id IS NULL AND po.procedure_date >= ? 
        AND c.domain_id = 'Procedure' AND (v.visit_occurrence_id IS NULL OR po.procedure_date <= v.visit_end_date)
    GROUP BY procedure_concept_id)) x
ORDER BY x.start_date;
"""

def _serialize_sequence(res):
    return '\t'.join([str(x[0]) + ',' + x[1].strftime('%Y-%m-%d') for x in res])

with open(path.join(dir_data, 'patient_code_sequences.txt'), 'w') as fh:
    print('patient sequence progress: ')
    n_patients = 0
    for index, row in df_persons.iterrows():
        # Display progress
        if index % 100000 == 0:
            percent = index / n_persons * 100
            elapsed_time = (time.time() - t1) / 60
            print(f'\t{percent:.02f}% - {elapsed_time:.01f} min')
        
        # Get a sequence of the patient's codes and dates
        pid = row['person_id']
        dob = row['birth_date']
        cursor.execute(sql, pid, dob, pid, dob, pid, dob)
        res = cursor.fetchall()
        
        # Write the sequence if it's not empty
        if res:
            sequence_str = _serialize_sequence(res)
            fh.write(str(pid) + '\t' + sequence_str + '\n')
            n_patients += 1

elapsed_time = (time.time() - t1) / 60
print(elapsed_time)